In [1]:
%load_ext autoreload
import datetime
# import ete3
import itertools
import json
import logging
import math
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import typing
# import re
import xml.etree.ElementTree as ET
# import xml.dom.minidom
# import xmlschema

In [17]:
%autoreload
from maxes.xes_loader2 import XesLoader, XesLog
# from maxes.xes_file import XesFile
# from maxes.analyze_xes import AnalyzeXes
# import maxes.analyze_sequence
# import maxes.serialization.serialize
# import maxes.graphs
import maxes.notebooks.utils
import maxes.utils

In [ ]:
maxes.notebooks.utils.init_notebook() # RUN ONLY ONCE

In [4]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

x = np.array([0, 1, 2, 3, 4, 5]).reshape(-1, 1)

# scaler = MinMaxScaler()
scaler = StandardScaler()
print(scaler.fit_transform(x))


data2 = np.array([-1, 6]).reshape(-1, 1)
print(scaler.transform(data2))

[[-1.46385011]
 [-0.87831007]
 [-0.29277002]
 [ 0.29277002]
 [ 0.87831007]
 [ 1.46385011]]
[[-2.04939015]
 [ 2.04939015]]


In [5]:
import maxes.notebooks.load_files


file_path = maxes.notebooks.utils.get_data_path("other/simple.xes")
log = maxes.notebooks.utils.load_xes(file_path)

# log = maxes.notebooks.load_files.load_activities_detailed_labour()
df = log.traces[1].df
df

,call centre,org:resource,time:timestamp,lifecycle:transition,concept:name,location,duration,outcome
0,Sydney,customer,1970-01-01 00:00:00+00:00,complete,incoming claim,NaN,NaN,NaN
1,NaN,Call Centre Agent Sydney,1970-01-01 00:00:00+00:00,start,S check if sufficient information is available,Sydney,NaN,NaN
2,NaN,Call Centre Agent Sydney,1970-01-01 00:00:21+00:00,complete,S check if sufficient information is available,Sydney,NaN,NaN
3,NaN,Call Centre Agent Sydney,1970-01-01 00:00:21+00:00,start,S register claim,NaN,NaN,NaN
4,NaN,Call Centre Agent Sydney,1970-01-01 00:01:54+00:00,complete,S register claim,NaN,NaN,NaN
5,NaN,Claims handler,1970-01-01 00:01:54+00:00,start,determine likelihood of claim,NaN,NaN,NaN
6,NaN,Claims handler,1970-01-01 00:02:05+00:00,complete,determine likelihood of claim,NaN,NaN,NaN
7,NaN,Claims handler,1970-01-01 00:02:05+00:00,start,assess claim,NaN,NaN,NaN
8,NaN,Claims handler,1970-01-01 00:04:09+00:00,complete,assess claim,NaN,NaN,NaN
9,NaN,Claims handler,1970-01-01 00:04:09+00:00,start,advise claimant on reimbursement,NaN,NaN,NaN


In [6]:
print("X values (values to predict from)")
print(log.df["concept:name"].value_counts())

print()
print()

print("Y values (values to be predicted)")
print(log.df["org:resource"].value_counts())

X values (values to predict from)
concept:name
end                                               7024
determine likelihood of claim                     5900
assess claim                                      4922
advise claimant on reimbursement                  3952
initiate payment                                  3952
close claim                                       3952
S check if sufficient information is available    3584
incoming claim                                    3512
B check if sufficient information is available    3440
B register claim                                  3050
S register claim                                  2850
Name: count, dtype: int64


Y values (values to be predicted)
org:resource
Claims handler                28578
Call Centre Agent Sydney       7168
Call Centre Agent Brisbane     6880
customer                       3512
Name: count, dtype: int64


In [7]:
from sklearn.preprocessing import OneHotEncoder

print(log.df["concept:name"][:5])

X = np.array(log.df["concept:name"]).reshape(-1, 1)
input_encoder_for_concept_name = OneHotEncoder()
X = input_encoder_for_concept_name.fit_transform(X)

print(input_encoder_for_concept_name.inverse_transform([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]))

print(X[:5].toarray())
# X

0                                    incoming claim
1    B check if sufficient information is available
2    B check if sufficient information is available
3                                  B register claim
4                                  B register claim
Name: concept:name, dtype: object
[['B check if sufficient information is available']]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [8]:

print(log.df["org:resource"][:5])

y = np.array(log.df["org:resource"]).reshape(-1, 1)
y = OneHotEncoder().fit_transform(y)

print(y[:5].toarray())

0                      customer
1    Call Centre Agent Brisbane
2    Call Centre Agent Brisbane
3    Call Centre Agent Brisbane
4    Call Centre Agent Brisbane
Name: org:resource, dtype: object
[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


In [9]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(hidden_layer_sizes=())
model.fit(X, y)


MLPClassifier(hidden_layer_sizes=())

In [10]:
model.predict([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]]).toarray()
model.predict([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]).toarray()

array([[1, 0, 0, 0]])

In [11]:
log.traces[0].df

,call centre,org:resource,time:timestamp,lifecycle:transition,concept:name,location,duration,outcome
0,Brisbane,customer,1970-01-01 00:00:00+00:00,complete,incoming claim,NaN,NaN,NaN
1,NaN,Call Centre Agent Brisbane,1970-01-01 00:00:00+00:00,start,B check if sufficient information is available,Brisbane,NaN,NaN
2,NaN,Call Centre Agent Brisbane,1970-01-01 00:00:41+00:00,complete,B check if sufficient information is available,Brisbane,NaN,NaN
3,NaN,Call Centre Agent Brisbane,1970-01-01 00:00:41+00:00,start,B register claim,NaN,NaN,NaN
4,NaN,Call Centre Agent Brisbane,1970-01-01 00:12:59+00:00,complete,B register claim,NaN,NaN,NaN
5,NaN,Claims handler,1970-01-01 00:47:04+00:00,start,determine likelihood of claim,NaN,NaN,NaN
6,NaN,Claims handler,1970-01-01 00:47:12+00:00,complete,determine likelihood of claim,NaN,NaN,NaN
7,NaN,Claims handler,1970-01-01 00:47:12+00:00,start,end,NaN,2832,not liable
8,NaN,Claims handler,1970-01-01 00:47:12+00:00,complete,end,NaN,2832,not liable


In [15]:
def normalize_series_minmax(series: pd.Series):
    return (series - series.min()) / (series.max() - series.min())

def normalize_series_mean(series: pd.Series):
    return (series - series.mean()) / series.std()

In [16]:
# Add "seconds since start" column

df["Seconds since start"] = (df["time:timestamp"] - df["time:timestamp"][0]).dt.total_seconds()
df["Seconds since previous event"] = (df["time:timestamp"] - df["time:timestamp"].shift(1)).dt.total_seconds()
df["Seconds since previous event (Normalized)"] = normalize_series_minmax(df["Seconds since previous event"])
df

,call centre,org:resource,time:timestamp,lifecycle:transition,concept:name,location,duration,outcome,Seconds since start,Seconds since previous event,Seconds since previous event (Normalized)
0,Sydney,customer,1970-01-01 00:00:00+00:00,complete,incoming claim,NaN,NaN,NaN,0.0,NaN,NaN
1,NaN,Call Centre Agent Sydney,1970-01-01 00:00:00+00:00,start,S check if sufficient information is available,Sydney,NaN,NaN,0.0,0.0,0.000000
2,NaN,Call Centre Agent Sydney,1970-01-01 00:00:21+00:00,complete,S check if sufficient information is available,Sydney,NaN,NaN,21.0,21.0,0.091703
3,NaN,Call Centre Agent Sydney,1970-01-01 00:00:21+00:00,start,S register claim,NaN,NaN,NaN,21.0,0.0,0.000000
4,NaN,Call Centre Agent Sydney,1970-01-01 00:01:54+00:00,complete,S register claim,NaN,NaN,NaN,114.0,93.0,0.406114
5,NaN,Claims handler,1970-01-01 00:01:54+00:00,start,determine likelihood of claim,NaN,NaN,NaN,114.0,0.0,0.000000
6,NaN,Claims handler,1970-01-01 00:02:05+00:00,complete,determine likelihood of claim,NaN,NaN,NaN,125.0,11.0,0.048035
7,NaN,Claims handler,1970-01-01 00:02:05+00:00,start,assess claim,NaN,NaN,NaN,125.0,0.0,0.000000
8,NaN,Claims handler,1970-01-01 00:04:09+00:00,complete,assess claim,NaN,NaN,NaN,249.0,124.0,0.541485
9,NaN,Claims handler,1970-01-01 00:04:09+00:00,start,advise claimant on reimbursement,NaN,NaN,NaN,249.0,0.0,0.000000


In [45]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPClassifier

def predict_categorical(
        log: XesLog,
        input_column_name: str,
        output_column_name: str
    ):

    input_encoder = OneHotEncoder()
    X_series = log.df[input_column_name]
    X_np = np.array(X_series).reshape(-1, 1)
    X = input_encoder.fit_transform(X_np)

    output_encoder = OneHotEncoder()
    y_series = log.df[output_column_name]
    y_np = np.array(y_series).reshape(-1, 1)
    y = output_encoder.fit_transform(y_np)

    model = MLPClassifier()
    model.fit(X, y)

    def pipeline(input):
        X = input_encoder.transform(input)
        y = model.predict(X)
        prediction = output_encoder.inverse_transform(y)
        return prediction.ravel()

    return model, input_encoder, output_encoder, pipeline


In [46]:
print(f"{datetime.datetime.now()} org_resource")
model_org_resource, input_encoder_org_resource, output_encoder_org_resource, predict_org_resource = predict_categorical(log, "concept:name", "org:resource")

print(f"{datetime.datetime.now()} location")
model_location, input_encoder_location, output_encoder_location, predict_location = predict_categorical(log, "concept:name", "location")

# print(f"{datetime.datetime.now()} duration")
# model_duration, input_encoder_duration, output_encoder_duration, predict_duration = predict_categorical(log, "concept:name", "duration")

print(f"{datetime.datetime.now()} outcome")
model_outcome, input_encoder_outcome, output_encoder_outcome, predict_outcome = predict_categorical(log, "concept:name", "outcome")

print(f"{datetime.datetime.now()} call_centre")
model_call_centre, input_encoder_call_centre, output_encoder_call_centre, predict_call_centre = predict_categorical(log, "concept:name", "call centre")

print(f"{datetime.datetime.now()} done")

2024-10-14 14:00:57.594760 org_resource
2024-10-14 14:01:03.742618 location
2024-10-14 14:01:07.064196 outcome
2024-10-14 14:01:10.468029 call_centre
2024-10-14 14:01:13.366960 done


In [30]:
from maxes.generators.xes_generator.xes_generator1 import XesGenerator1
# import maxes.generators.xes_generator.xes_generator1

generator = XesGenerator1(traces_count=1)
generator.fit(log)

In [66]:
generated_log = generator.generate()
generated_log._update_df()

generated_log.df["call centre"] = None
generated_log.df["org:resource"] = None
generated_log.df["location"] = None
generated_log.df["duration"] = None
generated_log.df["outcome"] = None

generated_log.df

,concept:name,lifecycle:transition,time:timestamp,call centre,org:resource,location,duration,outcome
0,incoming claim,complete,1970-01-01 07:37:07.372131+00:00,None,None,None,None,None
1,B check if sufficient information is available,start,1970-01-01 08:19:09.316028+00:00,None,None,None,None,None
2,B check if sufficient information is available,complete,1970-01-01 08:19:13.575879+00:00,None,None,None,None,None
3,B register claim,start,1970-01-01 08:19:12.429297+00:00,None,None,None,None,None
4,B register claim,complete,1970-01-01 08:21:07.122727+00:00,None,None,None,None,None
5,determine likelihood of claim,start,1970-01-01 08:21:51.230030+00:00,None,None,None,None,None
6,determine likelihood of claim,complete,1970-01-01 08:22:16.102977+00:00,None,None,None,None,None
7,assess claim,start,1970-01-01 08:22:14.621166+00:00,None,None,None,None,None
8,assess claim,complete,1970-01-01 08:30:06.736019+00:00,None,None,None,None,None
9,end,start,1970-01-01 08:30:09.131325+00:00,None,None,None,None,None


In [51]:
log.traces[0].df

,call centre,org:resource,time:timestamp,lifecycle:transition,concept:name,location,duration,outcome
0,Brisbane,customer,1970-01-01 00:00:00+00:00,complete,incoming claim,NaN,NaN,NaN
1,NaN,Call Centre Agent Brisbane,1970-01-01 00:00:00+00:00,start,B check if sufficient information is available,Brisbane,NaN,NaN
2,NaN,Call Centre Agent Brisbane,1970-01-01 00:00:41+00:00,complete,B check if sufficient information is available,Brisbane,NaN,NaN
3,NaN,Call Centre Agent Brisbane,1970-01-01 00:00:41+00:00,start,B register claim,NaN,NaN,NaN
4,NaN,Call Centre Agent Brisbane,1970-01-01 00:12:59+00:00,complete,B register claim,NaN,NaN,NaN
5,NaN,Claims handler,1970-01-01 00:47:04+00:00,start,determine likelihood of claim,NaN,NaN,NaN
6,NaN,Claims handler,1970-01-01 00:47:12+00:00,complete,determine likelihood of claim,NaN,NaN,NaN
7,NaN,Claims handler,1970-01-01 00:47:12+00:00,start,end,NaN,2832,not liable
8,NaN,Claims handler,1970-01-01 00:47:12+00:00,complete,end,NaN,2832,not liable


In [67]:
# concept_name = generated_log.df[["concept:name"]]
concept_name = np.array(generated_log.df["concept:name"]).reshape(-1, 1)

generated_log.df["org:resource"] = predict_org_resource(concept_name).ravel()
generated_log.df["location"] = predict_location(concept_name).ravel()
generated_log.df["call centre"] = predict_call_centre(concept_name).ravel()
generated_log.df["outcome"] = predict_outcome(concept_name).ravel()

generated_log.df

,concept:name,lifecycle:transition,time:timestamp,call centre,org:resource,location,duration,outcome
0,incoming claim,complete,1970-01-01 07:37:07.372131+00:00,Sydney,customer,NaN,None,NaN
1,B check if sufficient information is available,start,1970-01-01 08:19:09.316028+00:00,NaN,Call Centre Agent Brisbane,Brisbane,None,NaN
2,B check if sufficient information is available,complete,1970-01-01 08:19:13.575879+00:00,NaN,Call Centre Agent Brisbane,Brisbane,None,NaN
3,B register claim,start,1970-01-01 08:19:12.429297+00:00,NaN,Call Centre Agent Brisbane,NaN,None,NaN
4,B register claim,complete,1970-01-01 08:21:07.122727+00:00,NaN,Call Centre Agent Brisbane,NaN,None,NaN
5,determine likelihood of claim,start,1970-01-01 08:21:51.230030+00:00,NaN,Claims handler,NaN,None,NaN
6,determine likelihood of claim,complete,1970-01-01 08:22:16.102977+00:00,NaN,Claims handler,NaN,None,NaN
7,assess claim,start,1970-01-01 08:22:14.621166+00:00,NaN,Claims handler,NaN,None,NaN
8,assess claim,complete,1970-01-01 08:30:06.736019+00:00,NaN,Claims handler,NaN,None,NaN
9,end,start,1970-01-01 08:30:09.131325+00:00,NaN,Claims handler,NaN,None,processed


In [52]:
log.traces[0].df

,call centre,org:resource,time:timestamp,lifecycle:transition,concept:name,location,duration,outcome
0,Brisbane,customer,1970-01-01 00:00:00+00:00,complete,incoming claim,NaN,NaN,NaN
1,NaN,Call Centre Agent Brisbane,1970-01-01 00:00:00+00:00,start,B check if sufficient information is available,Brisbane,NaN,NaN
2,NaN,Call Centre Agent Brisbane,1970-01-01 00:00:41+00:00,complete,B check if sufficient information is available,Brisbane,NaN,NaN
3,NaN,Call Centre Agent Brisbane,1970-01-01 00:00:41+00:00,start,B register claim,NaN,NaN,NaN
4,NaN,Call Centre Agent Brisbane,1970-01-01 00:12:59+00:00,complete,B register claim,NaN,NaN,NaN
5,NaN,Claims handler,1970-01-01 00:47:04+00:00,start,determine likelihood of claim,NaN,NaN,NaN
6,NaN,Claims handler,1970-01-01 00:47:12+00:00,complete,determine likelihood of claim,NaN,NaN,NaN
7,NaN,Claims handler,1970-01-01 00:47:12+00:00,start,end,NaN,2832,not liable
8,NaN,Claims handler,1970-01-01 00:47:12+00:00,complete,end,NaN,2832,not liable


In [53]:
log.traces[1].df

,call centre,org:resource,time:timestamp,lifecycle:transition,concept:name,location,duration,outcome
0,Sydney,customer,1970-01-01 00:00:00+00:00,complete,incoming claim,NaN,NaN,NaN
1,NaN,Call Centre Agent Sydney,1970-01-01 00:00:00+00:00,start,S check if sufficient information is available,Sydney,NaN,NaN
2,NaN,Call Centre Agent Sydney,1970-01-01 00:00:21+00:00,complete,S check if sufficient information is available,Sydney,NaN,NaN
3,NaN,Call Centre Agent Sydney,1970-01-01 00:00:21+00:00,start,S register claim,NaN,NaN,NaN
4,NaN,Call Centre Agent Sydney,1970-01-01 00:01:54+00:00,complete,S register claim,NaN,NaN,NaN
5,NaN,Claims handler,1970-01-01 00:01:54+00:00,start,determine likelihood of claim,NaN,NaN,NaN
6,NaN,Claims handler,1970-01-01 00:02:05+00:00,complete,determine likelihood of claim,NaN,NaN,NaN
7,NaN,Claims handler,1970-01-01 00:02:05+00:00,start,assess claim,NaN,NaN,NaN
8,NaN,Claims handler,1970-01-01 00:04:09+00:00,complete,assess claim,NaN,NaN,NaN
9,NaN,Claims handler,1970-01-01 00:04:09+00:00,start,advise claimant on reimbursement,NaN,NaN,NaN


In [54]:
log.traces[2].df

,call centre,org:resource,time:timestamp,lifecycle:transition,concept:name,location,duration,outcome
0,Brisbane,customer,1970-01-01 00:00:45+00:00,complete,incoming claim,NaN,NaN,NaN
1,NaN,Call Centre Agent Brisbane,1970-01-01 00:00:45+00:00,start,B check if sufficient information is available,Brisbane,NaN,NaN
2,NaN,Call Centre Agent Brisbane,1970-01-01 00:01:12+00:00,complete,B check if sufficient information is available,Brisbane,NaN,NaN
3,NaN,Call Centre Agent Brisbane,1970-01-01 00:01:12+00:00,start,B register claim,NaN,NaN,NaN
4,NaN,Call Centre Agent Brisbane,1970-01-01 00:12:34+00:00,complete,B register claim,NaN,NaN,NaN
5,NaN,Claims handler,1970-01-01 00:14:22+00:00,start,determine likelihood of claim,NaN,NaN,NaN
6,NaN,Claims handler,1970-01-01 00:15:00+00:00,complete,determine likelihood of claim,NaN,NaN,NaN
7,NaN,Claims handler,1970-01-01 00:15:00+00:00,start,assess claim,NaN,NaN,NaN
8,NaN,Claims handler,1970-01-01 00:17:16+00:00,complete,assess claim,NaN,NaN,NaN
9,NaN,Claims handler,1970-01-01 00:17:16+00:00,start,initiate payment,NaN,NaN,NaN
